<a href="https://colab.research.google.com/github/neon-aiart/chirp-whisper-link/blob/main/chirp-whisper-link%20v4.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🐦 Chirping Whisper Link
APP_VERSION = '4.5'
# @markdown ---
model_type = "turbo" # @param ["tiny", "base", "small", "medium", "large-v3", "turbo"]
initial_prompt = "" # @param {type:"string"}
mode = "Upload" # @param ["Upload", "YouTube"]
youtube_url = "Youtube URL" # @param {type:"string"}
language = "ja"
condition_on_previous_text = False
execute_file_exists = False  # @param {type:"boolean"}
records_text_download = False  # @param {type:"boolean"}
# @markdown ---

import sys

print(f"🐦 Chirping Whisper Link v{APP_VERSION}")

# 1. ライブラリ準備
if 'faster_whisper' not in sys.modules:
    print("📦 Faster-Whisperをインストール中...")
    !pip install faster-whisper yt-dlp -q

import os, time, torch, gc, warnings
from faster_whisper import WhisperModel
from datetime import datetime
from google.colab import files, output
from IPython.display import clear_output, Audio, display
import numpy as np

# Hugging Faceのトークン警告を非表示にする
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub")

# タイムゾーン設定
browser_lang = output.eval_js('document.documentElement.lang')

if 'ja' in browser_lang:
    language = "ja"
    os.environ['TZ'] = 'Asia/Tokyo'
else:
    language = "en"
    os.environ['TZ'] = 'UTC'

time.tzset() # タイムゾーンの設定を反映

# 2. モデルのロード (Faster版)
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

if 'model' not in globals() or current_model_type != model_type:
    print(f"📦 Whisperモデル({model_type})を読み込み中... (数分かかります)")
    # Faster-Whisper特有の呼び出し方
    model = WhisperModel(model_type, device=device, compute_type=compute_type)
    current_model_type = model_type
    clear_output()
    print(f"✅ モデル({model_type})のロードが完了しました。")
else:
    print(f"⚡ モデル({model_type})はロード済みです。")

# 3. メモリ掃除
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 4. 入力ソースの選択
file_name = None

if mode == "YouTube":
    if youtube_url == "Youtube URL" or not youtube_url.startswith("http"):
        print("⚠️ 有効なYouTube URLを入力してください。")
    else:
        import yt_dlp
        print("📺 YouTubeから音声を抽出中...")
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'youtube_audio.%(ext)s',
            'noplaylist': True,
            'quiet': True,
            'no_warnings': True,
        }
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(youtube_url, download=True)
                file_name = ydl.prepare_filename(info)

                # .webm などに変わっている可能性を考慮
                if not os.path.exists(file_name):
                    file_name = info.get('requested_downloads', [{}])[0].get('filepath', file_name)
        except Exception as e:
            print(f"❌ YouTube動画のダウンロードに失敗しました: {e}")
            file_name = None # Ensure file_name is None if download fails
else:
    # --- 📂 ローカルファイルのスマート選択 ---
    valid_extensions = ('.mp3', '.mp4', '.wav', '.m4a', '.webm', '.ogg', '.flac')
    # フォルダ内の対象ファイルをリストアップ（更新日時が新しい順）
    local_files = sorted(
        [f for f in os.listdir('.') if f.endswith(valid_extensions)],
        key=os.path.getmtime, reverse=True
    )

    if local_files and execute_file_exists:
        # ファイルが存在し、かつ execute_file_exists が True の場合のみ既存ファイルを使用
        file_name = local_files[0]
    else:
        print("📂 ローカルファイルをアップロードしてください")
        uploaded = files.upload()
        if uploaded:
            file_name = list(uploaded.keys())[0]

# 5. 文字起こし実行
if file_name:
    print(f"\n🚀 実行中 (Fasterモード): {file_name}")

    # 1. 最初に音声の総時間を取得（バーの最大値にするため）
    segments, info = model.transcribe(
        file_name,
        initial_prompt=initial_prompt,
        language=None,
        condition_on_previous_text=condition_on_previous_text,
        beam_size=5,
        # --- ここからが精度のための追加設定 ---
        vad_filter=True,                  # 余計なノイズや無音をカット
        vad_parameters=dict(min_silence_duration_ms=500),
        no_speech_threshold=0.6,          # 喋っていない場所を無理に訳さない
        compression_ratio_threshold=2.4,  # 変なループ（同じ言葉の繰り返し）を防ぐ
        log_prob_threshold=-1.0,          # 自信がない時に適当なことを言わせない
        max_new_tokens=128,               # 1つの字幕の最大文字数を制限
        repetition_penalty=1.2,           # ループ（繰り返し）をより厳しく抑制
    )

    # 2. tqdmで進捗バーを準備
    from tqdm.notebook import tqdm
    pbar = tqdm(total=info.duration, unit="sec") # 総秒数をゴールに設定
    last_t = 0

    results_list = []
    full_text = ""

    for segment in segments:
        results_list.append(segment)
        full_text += segment.text

        # 3. 進捗バーを更新
        pbar.update(segment.end - last_t)
        last_t = segment.end

    pbar.close() # 終わったら閉じる

    # 保存
    def format_timestamp(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        millis = int((seconds - int(seconds)) * 1000)
        return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

    # SRTとLOGの作成
    srt_output = ""
    log_output = ""
    for i, seg in enumerate(results_list):
        s_time = format_timestamp(seg.start)
        e_time = format_timestamp(seg.end)
        srt_output += f"{i + 1}\n{s_time} --> {e_time}\n{seg.text.strip()}\n\n"
        log_output += f"[{s_time[:8]}] {seg.text.strip()}\n"

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = os.path.splitext(os.path.basename(file_name))[0]
    output_name_srt = f"{base}_{timestamp}.srt"
    with open(output_name_srt, "w", encoding="utf-8") as f: f.write(srt_output)
    output_name_log = f"{base}_{timestamp}.txt"
    with open(output_name_log, "w", encoding="utf-8") as f: f.write(log_output)

    # 結果表示
    clear_output()
    print(f"✅ 完了！: {file_name}")

    # --- 完了通知音（きらめき音） ---
    sample_rate = 44100 # Hz

    # 音を構成する要素の数
    num_sparkles = 3
    # 各きらめき音の長さ
    sparkle_duration = 0.08 # 秒
    # 周波数の全体的な開始範囲と終了範囲
    base_start_frequency = 2000 # Hz
    base_end_frequency = 4000 # Hz (全体的に上昇するような効果)
    # 急速な減衰率
    decay_rate = 20

    all_data = []
    for i in range(num_sparkles):
        # 各きらめき音のタイムベクトル
        t_sparkle = np.linspace(0, sparkle_duration, int(sparkle_duration * sample_rate), endpoint=False)

        # 各きらめき音の開始周波数と終了周波数を計算
        # これにより、全体のきらめき音が上昇するシーケンスになります
        f_start_current_sparkle = base_start_frequency + (base_end_frequency - base_start_frequency) * (i / num_sparkles)
        f_end_current_sparkle = base_start_frequency + (base_end_frequency - base_start_frequency) * ((i + 1) / num_sparkles)

        # 各きらめき音内でわずかな上昇スイープを導入し、きらめき効果を強調
        # 実際の周波数は f_start_current_sparkle から f_end_current_sparkle まで変化します
        instantaneous_frequency = np.linspace(f_start_current_sparkle, f_end_current_sparkle, len(t_sparkle))

        # 波形を生成
        sparkle_wave = np.sin(2 * np.pi * instantaneous_frequency * t_sparkle)

        # 指数関数的減衰を適用
        envelope_sparkle = np.exp(-decay_rate * t_sparkle)
        sparkle_data = sparkle_wave * envelope_sparkle

        all_data.append(sparkle_data)

        # 各きらめき音の間に非常に短い無音を挿入（最後の音以外）
        # これにより、それぞれの音がはっきりと聞こえ、混ざり合うのを防ぎます
        if i < num_sparkles - 1:
            silence_duration = 0.02 # 秒
            silence = np.zeros(int(silence_duration * sample_rate))
            all_data.append(silence)

    data = np.concatenate(all_data)

    # ボリュームを正規化（クリッピング防止と音量調整）
    # データが空の場合はエラーにならないように処理
    data = data / np.max(np.abs(data)) * 0.8 if len(data) > 0 else np.array([0.0])

    display(Audio(data, rate=sample_rate, autoplay=True))

    # 6. ダウンロード
    print("📥 ファイルをダウンロードしています...")
    # files.download(output_name_txt)
    files.download(output_name_srt)
    if records_text_download:
        files.download(output_name_log)

    print("\n" + "!"*40)
    print("⚠️ ATTENTION: PLEASE DISCONNECT RUNTIME")
    print("⚠️ 接続解除を忘れないでください！残り時間が削られます。")
    print("!"*40)

    # YouTube用の一時ファイル削除
    if mode == "YouTube" and file_name and os.path.exists(file_name):
        os.remove(file_name)
else:
    print("⚠️ 処理がキャンセルされました。")